In [324]:
# For Building recommender systems
!pip install scikit-surprise

In [325]:
import pandas as pd
import numpy as np
import joblib
from surprise import SVD, Reader, Dataset, accuracy

In [326]:
news_df = pd.read_csv('/content/news.csv')
rec_items_df = pd.read_csv('/content/rec_items.csv')
rec_feedback_df = pd.read_csv('/content/rec_feedback.csv')
users_df = pd.read_csv('/content/users.csv')
rec_users_df = pd.read_csv('/content/rec_users.csv')

In [327]:
# Load the CBF model (TF-IDF and Cosine Similarity Matrix)
tfidf = joblib.load('/content/tfidf_vectorizer.pkl')
cosine_sim = joblib.load('/content/cosine_similarity_matrix.pkl')

# Load the CF model (e.g., a pre-trained collaborative filtering model)
cf_model = joblib.load('/content/svd_recommender_model.pkl')

In [328]:
# Define custom interaction weights
interaction_weights = {'click': 1, 'read': 2, 'like': 3}

# Map interaction types to their corresponding weights
rec_feedback_df['rating'] = rec_feedback_df['feedback_type'].map(interaction_weights)

rec_feedback_df['rating'] = rec_feedback_df['rating'].fillna(0)

In [329]:
# Function to get CBF (Content-Based Filtering) recommendations
def get_cbf_recommendations(user_id, top_n=5):
    # Get the articles the user has interacted with
    user_interactions = rec_feedback_df[rec_feedback_df['user_id'] == user_id]
    interacted_article_ids = user_interactions['item_id'].unique()

    if len(interacted_article_ids) == 0:
        return "No interactions found for this user."

    # Get indices of the interacted articles in the news_df
    interacted_article_indices = news_df[news_df['id'].isin(interacted_article_ids)].index.tolist()

    if len(interacted_article_indices) == 0:
        return "No articles found for user interactions."

    # Get similarity scores for the interacted articles
    sim_scores = []
    for idx in interacted_article_indices:
        sim_scores.extend(list(enumerate(cosine_sim[idx])))

    # Sort articles by similarity score
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Collect recommended articles (avoid recommending already interacted articles)
    recommended_articles = []
    for score in sim_scores:
        article_idx = score[0]
        if news_df.iloc[article_idx]['id'] not in interacted_article_ids:
            recommended_articles.append(news_df.iloc[article_idx][['id', 'title', 'description']])
        if len(recommended_articles) == top_n:
            break

    return pd.DataFrame(recommended_articles)

In [330]:
# Function to get CF (Collaborative Filtering) recommendations
def get_cf_recommendations(user_id, top_n=5):
    # Get the user interaction data
    user_interactions = rec_feedback_df[rec_feedback_df['user_id'] == user_id]

    if user_interactions.empty:
        return "No interactions found for this user."

    # Prepare the dataset for the SVD model using interaction weights
    reader = Reader(rating_scale=(0, 3))  # Use the custom rating scale (0 to 3 based on your weights)
    data = Dataset.load_from_df(rec_feedback_df[['user_id', 'item_id', 'rating']], reader)
    trainset = data.build_full_trainset()

    # Get all items in the dataset
    all_items = news_df['id'].tolist()

    # Get predicted ratings for each item for the given user
    predictions = []
    for item_id in all_items:
        prediction = cf_model.predict(user_id, item_id)  # Predict the rating for user and item
        predictions.append((item_id, prediction.est))  # Collect item and predicted rating

    # Sort predictions based on predicted rating in descending order
    predictions.sort(key=lambda x: x[1], reverse=True)

    # Get the top-N recommended items
    recommended_articles = []
    for item_id, _ in predictions[:top_n]:
        article = news_df[news_df['id'] == item_id][['id', 'title', 'description']].iloc[0]
        recommended_articles.append(article)

    return pd.DataFrame(recommended_articles)

In [331]:
# Hybrid Recommendation Function (combining CF and CBF)
def get_hybrid_recommendations(user_id, top_n=5, alpha=0.5):
    """
    Combine CF and CBF recommendations with a weight parameter alpha.
    alpha: weight to combine CF and CBF (0.5 for equal weight).
    """
    # Get CBF recommendations
    cbf_recs = get_cbf_recommendations(user_id, top_n)

    # If no CBF recommendations, return CF recommendations
    if isinstance(cbf_recs, str):
        return get_cf_recommendations(user_id, top_n)

    # Get CF recommendations
    cf_recs = get_cf_recommendations(user_id, top_n)

    # Combine recommendations
    # Combine the top recommendations from CBF and CF
    # Add weight to the CBF and CF recommendations
    cbf_recs['score'] = alpha  # Assuming equal weight, you can adjust 'alpha' for tuning
    cf_recs['score'] = 1 - alpha  # Complementary weight for CF

    # Merge both recommendations
    combined_recommendations = pd.concat([cbf_recs, cf_recs], ignore_index=True)
    combined_recommendations = combined_recommendations.drop_duplicates(subset='id', keep='first')

    # Sort the combined recommendations based on their scores
    combined_recommendations = combined_recommendations.sort_values(by='score', ascending=False)

    # Get the top-N recommendations
    return combined_recommendations.head(top_n)

In [332]:
user_id = 2329
hybrid_recommendations = get_hybrid_recommendations(user_id, top_n=5, alpha=0.5)

# Display the hybrid recommendations
print(hybrid_recommendations)

      id                                              title  \
0  23269  ලාංකිකයින් 3065ක් අමෙරිකාවෙන් පිටුවහල් කිරීමට ...   
1  23484            ඉන්ධන සැපයූ සමාගමක් ලංකාවෙන් ඉවත්ව යයි.   
2  23848                   ට්‍රම්ප්ගෙන් 25%ක අලුත් බද්දක්\n   
3  23521           සංචාරකයින්ගේ පැමිණීමේ වාර්තාගත වැඩිවීමක්   
4  23826        ජනපතිගේ ඩුබායි නිල සංචාරයේ දෙවැනි දිනය අදයි   

                                         description  score  
0  නව අමෙරිකානු පරිපාලනය විසින් පිටුවහල් කිරීමට න...    0.5  
1  මෙරට ඉන්ධන වෙලදාම සදහා පැමිණි ඕස්ට්‍රේලියාවේ ...    0.5  
2  අමෙරිකාවට ආනයනය කරන සියලු‍ම වානේ සහ ඇලු‍මිනියම...    0.5  
3  මෙරටට පැමිණි සංචාරකයින්ගේ වාර්තාගත වැඩි වීමක් ...    0.5  
4   ජනාධිපති අනුර කුමාර දිසානායක මහතාගේ එක්සත් අර...    0.5  


In [333]:
user_id = 645
hybrid_recommendations = get_hybrid_recommendations(user_id, top_n=5, alpha=0.5)

# Display the hybrid recommendations
print(hybrid_recommendations)

      id                                              title  \
0  23854  පරිභෝජනයට නුසුදුසු කඩල පරිප්පු වෙළෙඳ පොළට නිකු...   
1  23852        CID වැඩවලට අලුත් විමර්ශන කොට්ඨාසයක් එක්වෙයි   
2  23751              හිටපු අගමැති ඛාන් වෙනුවෙන් විරෝධතාවක්   
3  23864  ක‍්‍රීඩා නියෝජ්‍ය ඇමති පවුල පිටින් ඕස්ට්‍රේලිය...   
4  23701  හිටපු සුරාබදු කොමසාරිස් ජෙනරාල්වරයාට එරෙහිව ගො...   

                                         description  score  
0  ගුල්ලන් සහිත මනුෂ්‍යය පරිභෝජනයට නුසුදුසු කඩල ප...    0.5  
1  අපරාධ පරීක්ෂණ දෙපාර්තමේන්තුව යටතේ ඇති නොයෙකුත්...    0.5  
2  විවිධ චෝදනා රැසකට වරදකරු කර ඇති පාකිස්තානයේ හි...    0.5  
3  ක්‍රීඩා හා තරුණ කටයුතු නියෝජ්‍ය ඇමති සුගත් තිල...    0.5  
4  අධිකරණයට අපහාස කළේ යැයි චෝදනා එල්ලවූ හිටපු සුර...    0.5  


In [335]:
user_id = 678
hybrid_recommendations = get_hybrid_recommendations(user_id, top_n=5, alpha=0.5)

# Display the hybrid recommendations
print(hybrid_recommendations)

      id                                              title  \
0  23762  යළි විදුලි කප්පාදු කාල සටහනක් එයි. - නොරොච්චලේ...   
1  23804        භික්ෂුවකට අකැප දේ කළ නා හිමි නමක් සිපිරිගෙට   
2  23914              නොරොච්චෝලේ අද පනගන්වන්න දැඩි උත්සහයක්   
3  23796  නොරොච්චෝල ON වෙන තුරු විදුලි කප්පාදුවක්. සැලසු...   
4  23859                 අදත් පැය එකහමාරක විදුලි කප්පාදුවක්   

                                         description  score  
0  ඉදිරි දින කිහිපයක් සඳහා විදුලිය කප්පාදු කිරීමට...    0.5  
1  නීතිවිරෝධී මත්ද්‍රව්‍යයක් ලෙස සැලකෙන හෙරොයින් ...    0.5  
2  බිඳවැටුණු නොරොච්චෝලේ විදුලි බලාගාරය යළි යථා තත...    0.5  
3  අද (10) සහ හෙට (11) දින දෙකේදී පැය එකහමාර බැගි...    0.5  
4  මේ අතර අද (11) දිනයේ ත් දිවයින පුරා මිනිත්තු 9...    0.5  
